## Imports

In [ ]:
# Standard library imports
import os
import pickle
import random
import time
import warnings
import glob
import shutil

# Third-party imports
import cv2
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from patchify import patchify, unpatchify
from PIL import Image
from skimage.io import imread, imshow
from skimage.transform import resize
import nbimporte
from unet_model import UNET
from tqdm import tqdm

# Setting up matplotlib to work interactively in a Jupyter environment
%matplotlib inline

# Setting the seed for reproducibility
seed_value = 42
random.seed(seed_value)  # For Python's built-in random module
np.random.seed(seed_value)  # For NumPy
tf.random.set_seed(seed_value)  # For TensorFlow

# Don't show warning messages
warnings.filterwarnings('ignore')


In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:

      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    

## Load data for self-supervised learning

In [ ]:
unlabeled =r'< PATH_TO_SELF_SUPERVISED_DATA >'

In [ ]:
# this function used to crop the unlabeled images along side their pseudo labels and store both mask and image along side the training dataset.
def process_alter(image, label, patches, patch_size, count_, full_model,iter_):

    for ptch in range(patches):
#         y = np.random.randint(0,IMG_HEIGHT - patch_size+1)
#         x = np.random.randint(0,IMG_WIDTH - patch_size+1)
#         im_crop = image[ y:y+patch_size, x:x+patch_size]
#         lab_crop = label[ y:y+patch_size, x:x+patch_size]
        
        im_crop = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH))
        lab_crop = cv2.resize(label, (IMG_HEIGHT, IMG_WIDTH))
#         if (np.mean(lab_crop)!=0):
        if full_model:
            cv2.imwrite(x_pseudo_pth+str(count_)+'_'+ str(ptch)+ '_'+ str(iter_)+'.png', im_crop)
            cv2.imwrite(y_pseudo_pth+'M'+str(count_)+'_'+ str(ptch)+ '_'+ str(iter_)+'.png', lab_crop*255)
        else:
            cv2.imwrite(x_pseudo_pth+str(count_)+'_'+ str(ptch)+ '_'+ str(iter_)+'.png', im_crop)
            cv2.imwrite(y_pseudo_pth+str(count_)+'_'+ str(ptch)+ '_'+ str(iter_)+'.png', lab_crop*255)
        
  

In [ ]:
# only select patches with atleast ones    ### TRAIN DATASET###
# save_data_pth = r'F:\semi_P1\Unlabeled__Images\\'


In [ ]:
# Patch_Generator(data_path, '', save_data_pth, '', include_large_img=include_full_size_img, with_labels=False)

In [ ]:
def entropy_error(mask):
    return -1 * np.sum(mask * np.log2(mask+1e-7))


### Initialize The Model

In [ ]:
from unet_model import UNET

In [ ]:
model_ss = UNET(IMG_HEIGHT)
model_ss.exec_()

In [ ]:
sim, acc_, f1_, rec, preci, std_sim, std_acc, std_f1, std_rec, std_preci = Test_(base_ptch_1024.model, test_list_large, test_gen_large)
print(f"Jaccard sim: {round(sim, 3)} ± {round(std_sim, 3)}, Accuracy: {round(acc_, 3)} ± {round(std_acc, 3)}, Precision: {round(preci, 3)} ± {round(std_preci, 3)}, F1-score: {round(f1_, 3)} ± {round(std_f1, 3)}, Recall: {round(rec, 3)} ± {round(std_rec, 3)}")


In [ ]:
x_unlabeled = []
for img_path in sorted(glob.glob(os.path.join(r'F:\semi_\Final\new_unlableled_images\\*.*'))):

    x_unlabeled.append(img_path)
len(x_unlabeled)

In [ ]:
def sorting_entrop(model, x_unlabeled, mean, std, IMG_HEIGHT):
    print("Calculating Entropies and Sorting")
    entropies_unlab = []
    un_lab_images_fn = []
    for unlab_img_fn in tqdm(x_unlabeled):
        image = cv2.imread(unlab_img_fn,0)
        image = cv2.resize(image, (IMG_HEIGHT, IMG_HEIGHT))/255
     
        image = (image - mean)/ std 
            
       

        image = np.array(preprocess(image, True)).reshape(1,IMG_HEIGHT,IMG_WIDTH,1 )
        image = image.reshape(1,IMG_HEIGHT,IMG_HEIGHT,1 )
        pred = model.predict(image,verbose=0)
        pred_ = (pred >= 0.5).astype(np.uint8)
        area = np.sum(pred_)/(IMG_HEIGHT*IMG_WIDTH)
        
        
        
        if area > entr_area_th:


            un_lab_images_fn.append(unlab_img_fn)
            entropies_unlab.append(entropy_error(pred))
        del image, pred
    
    print("Sorting")
    combine_list = list(zip(entropies_unlab, un_lab_images_fn))
    sorted_list = sorted(combine_list, key=lambda x:x[0])
    x_unlabeled_sort = [x[1] for x in sorted_list]
    entr_unlabeled_sort = [x[0] for x in sorted_list]
    
    del combine_list, sorted_list, entropies_unlab
   
    
    return x_unlabeled_sort
x_unlab_sort__main = sorting_entrop(model_ss.model,x_unlabeled, mean, std, IMG_HEIGHT)
print(len(x_unlab_sort__main))



## Training Model

In [ ]:
checkpointer = tf.keras.callbacks.ModelCheckpoint('20%', verbose=1, save_best_only=True)

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='logs')
]
def Self_SS(model, x_unlabeled_sort, mean, std, IMG_HEIGHT, train_data_pth, train_mask_pth, mean_ch = False, main_test=False, btch_sz= 32, full_model=False, iter_ = 0):
    

    X_new_filter = x_unlabeled_sort[: int(len(x_unlabeled_sort)*selected_portion)]   
    
    
    print("Generating Pseudo labels")

    for i, n_img_fn in enumerate(X_new_filter):
        image_org = cv2.imread(n_img_fn,0)
        image = cv2.resize(image_org, (IMG_HEIGHT, IMG_HEIGHT))/255
        image = (image-mean)/std
        image = image.reshape(1,IMG_HEIGHT,IMG_HEIGHT,1 )
        pred = model.predict(image,verbose=0).reshape(IMG_HEIGHT,IMG_HEIGHT)
        pred = cv2.erode(pred.reshape(IMG_HEIGHT,IMG_HEIGHT), kernel, iterations = 1)
        pseudo_lab = np.where(pred>0.5,1,0).astype(np.uint8)

        process_alter(image_org,pseudo_lab , no_of_pathces, altr_ptch_sz, i, full_model, iter_)
        
        
    print("Training ...")
    train_list = os.listdir(x_pseudo_pth)
    t_list = train_list[0:int((1-val_set)*len(train_list))]
    v_list = train_list[int((1-val_set)*len(train_list)):]
    
   
 
    train_gen = image_data_generator(t_list,x_pseudo_pth,y_pseudo_pth, batch_size=btch_sz, data_aug=True, main_test=main_test )
    val_gen = image_data_generator(v_list,x_pseudo_pth,y_pseudo_pth, batch_size=btch_sz, data_aug=False,main_test=main_test)
    
    

    results = model.fit_generator(train_gen, epochs=20, steps_per_epoch=int(len(t_list)/btch_sz),
                                      validation_data=val_gen, validation_steps=int(len(v_list)/btch_sz),
                                      callbacks=callbacks)
    
    
    
 

In [ ]:
def sec_to_(duration):
    hours = duration // 3600
    minutes = (duration - (hours * 3600)) // 60
    seconds = duration - ((hours * 3600) + (minutes * 60))
    msg = f'training elapsed time was {str(hours)} hours, {minutes:4.1f} minutes, {seconds:4.2f} seconds)'
    return msg # print out inferenceduration time

In [ ]:

train_data_pth = r'< PATH_TO_TRAIN_DATA >'   
train_mask_pth = r'< PATH_TO_TRAIN_MASK >'

In [ ]:

selected_portion = 0.03
iteration_ss = int(1/selected_portion)
iteration_ss = 10
entr_area_th = 0.02
val_set = 0.2

x_pseudo_pth = r'< PATH_FOR_X >'  
y_pseudo_pth =r'< PATH_FOR_Y >'  
kernel = np.ones((5,5), np.uint8)

if os.path.exists(x_pseudo_pth):
    shutil.rmtree(x_pseudo_pth)
    shutil.rmtree(y_pseudo_pth)
    
print("Copying folder")
shutil.copytree(train_data_pth, x_pseudo_pth)
shutil.copytree(train_mask_pth, y_pseudo_pth)




## Sorting - May take Time

In [ ]:
x_unlab_sort__main = sorting_entrop(model_ss.model,x_unlabeled, mean, std, IMG_HEIGHT)
print(len(x_unlab_sort__main))

In [ ]:
import pickle
with open('unlab_list5.txt', 'wb') as file:
    # Dump the list into the file
    pickle.dump(x_unlab_sort__main, file)

In [ ]:
with open('unlab_list5.txt', 'rb') as file:
    # Load the list from the file
    x_unlab_sort__main = pickle.load(file)
  

In [ ]:

for iter_ in range(iteration_ss):
    
    print(f"------Self Supervised Epoch: {iter_}/{iteration_ss}-----")
                            
    
    now = time.time()
    

    no_of_pathces = 2
    altr_ptch_sz = 856
    IMG_HEIGHT = 1024
    IMG_WIDTH=1024
    Self_SS(model_ss.model, x_unlab_sort__main, mean, std, IMG_HEIGHT, train_data_pth, train_mask_pth, mean_ch = False, main_test=False, btch_sz=8,full_model=False, iter_=iter_ )
    
    
    
    
    x_unlab_sort__main = x_unlab_sort__main[int(len(x_unlab_sort__main)*selected_portion):] 
    print(len(x_unlab_sort__main))
    end_r = time.time()
    inf_t = int(end_r - now)
    print(sec_to_(inf_t))
    print("Testing")
    
    sim, acc_, f1_, rec, preci, std_sim, std_acc, std_f1, std_rec, std_preci = Test_(base_ptch_1024.model, test_list_large, test_gen_large)
    print(f"Jaccard sim: {round(sim, 3)} ± {round(std_sim, 3)}, Accuracy: {round(acc_, 3)} ± {round(std_acc, 3)}, Precision: {round(preci, 3)} ± {round(std_preci, 3)}, F1-score: {round(f1_, 3)} ± {round(std_f1, 3)}, Recall: {round(rec, 3)} ± {round(std_rec, 3)}")




#     if (jaccard_sim > prev_sim):
#     selected_portion += selected_portion

    good_model = f'model_100%_ss_iter_v233{str(iter_)}.h5'
    model_ss.model.save_weights(good_model)
#     prev_sim = jaccard_sim
#     else:

#         base_ptch_512.load_weights(good_model)
#         print(f"Loaded model with this sim::{prev_sim}")
